Wege um die Effizient der Schachbretterkennugn zu erhöhen:
1.

In [ ]:
def edge_detect(file):
    image = cv2.imread(file)
    im_bw = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    (found, img) = findChessCorners(im_bw, 7, 7)
    if not found:
        return None
    return img

In [ ]:
def findChessCorners(img,nx, ny):
    #(found, corners) = cv2.findChessboardCorners(img, (nx, ny), cv2.CALIB_CB_ADAPTIVE_THRESH | cv2.CALIB_CB_FAST_CHECK | cv2.CALIB_CB_NORMALIZE_IMAGE)
    (found, corners) = cv2.findChessboardCorners(img, (nx, ny), None)
    if found:
        # Draw and display the corners
        cv2.drawChessboardCorners(img, (nx, ny), corners, found)

        #take corners and extrapolate in each direction to make out outer corners
        return True, img
    else:
        return False, img

In [ ]:
def extrapolate(cvCorners):

    return expCorners

In [ ]:
import glob
import cv2
import numpy as np
chess_images = glob.glob('../testFiles/emptyChessBoard/*.jpg')
# Select any index to grab an image from the list
for x, img in enumerate(chess_images):
    image = edge_detect(img)
    result_name = 'result/ChessBoard'+str(x)+'.jpg'
    if image is not None:
        cv2.imwrite(result_name, image)

In [ ]:
import numpy as np
import cv2
import glob
import yaml
#import pathlib

# termination criteria
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
objp = np.zeros((7*7,3), np.float32)
objp[:,:2] = np.mgrid[0:7,0:7].T.reshape(-1,2)
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space
imgpoints = [] # 2d points in image plane.

images = glob.glob('../datasets/Chess-Pieces_big/test/images/*.jpg')

# path = 'results'
# pathlib.Path(path).mkdir(parents=True, exist_ok=True)

found = 0
for fname in images:  # Here, 10 can be changed to whatever number you like to choose
    img = cv2.imread(fname) # Capture frame-by-frame
    #print(images[im_i])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    ret, corners = cv2.findChessboardCorners(gray, (7,7), None)
    # If found, add object points, image points (after refining them)
    if ret == True:
        objpoints.append(objp)   # Certainly, every loop objp is the same, in 3D.
        corners2 = cv2.cornerSubPix(gray,corners,(11,11),(-1,-1),criteria)
        imgpoints.append(corners2)
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7,7), corners2, ret)
        found += 1
        cv2.imshow('img', img)
        cv2.waitKey(500)
        # if you want to save images with detected corners
        # uncomment following 2 lines and lines 5, 18 and 19
        # image_name = path + '/calibresult' + str(found) + '.png'
        # cv2.imwrite(image_name, img)

print("Number of images used for calibration: ", found)

# When everything done, release the capture
# cap.release()
cv2.destroyAllWindows()

# calibration
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

# transform the matrix and distortion coefficients to writable lists
data = {'camera_matrix': np.asarray(mtx).tolist(),
        'dist_coeff': np.asarray(dist).tolist()}

# and save it to a file
with open("calibration_matrix.yaml", "w") as f:
    yaml.dump(data, f)

# done